In [169]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

df = pd.read_csv('..\data\df_merged.csv',index_col=0)
df = df.fillna(method='ffill')
df = df.set_index('日期')
df = df.astype('float64')
display(df.head())
display(df.tail())

,H-110 COT,H-120 COT,H-130 COT,H-140 COT,H-150 COT,H-160 COT,H-170 COT,H-180 COT,H-190 COT,n-P值,i-P值,n+i P值,Aromatics,N,O,乙烯產量,丙烯產量
日期,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,824.37,824.76,824.67,237.69,825.08,830.80,814.64,824.68,825.01,40.3,39.93,80.23,3.88,14.8,0.84,2031.233,959.48
2020-01-01 00:10:00,825.28,825.38,825.61,252.83,825.51,834.09,815.16,825.39,825.42,40.3,39.93,80.23,3.88,14.8,0.84,2031.233,959.48
2020-01-01 00:20:00,824.67,824.87,824.47,265.87,824.03,840.08,814.72,824.52,824.96,40.3,39.93,80.23,3.88,14.8,0.84,2031.233,959.48
2020-01-01 00:30:00,823.62,824.49,824.76,275.72,823.62,845.13,814.67,824.05,824.67,40.3,39.93,80.23,3.88,14.8,0.84,2031.233,959.48
2020-01-01 00:40:00,825.00,824.92,825.07,285.64,825.80,845.21,815.50,825.41,825.17,40.3,39.93,80.23,3.88,14.8,0.84,2031.233,959.48


,H-110 COT,H-120 COT,H-130 COT,H-140 COT,H-150 COT,H-160 COT,H-170 COT,H-180 COT,H-190 COT,n-P值,i-P值,n+i P值,Aromatics,N,O,乙烯產量,丙烯產量
日期,,,,,,,,,,,,,,,,,
2020-12-31 23:20:00,818.15,817.72,817.37,818.09,817.59,816.89,817.96,817.18,478.11,38.02,39.70,77.72,3.25,17.18,0.95,2105.759,1077.787
2020-12-31 23:30:00,817.00,817.63,816.80,817.88,817.20,818.47,817.68,817.58,469.84,38.02,39.70,77.72,3.25,17.18,0.95,2105.759,1077.787
2020-12-31 23:40:00,817.50,817.39,817.53,818.45,817.37,817.88,817.68,817.61,463.99,38.02,39.70,77.72,3.25,17.18,0.95,2105.759,1077.787
2020-12-31 23:50:00,817.44,818.41,817.86,816.79,817.16,817.64,817.94,817.20,458.82,38.02,39.70,77.72,3.25,17.18,0.95,2105.759,1077.787
2021-01-01 00:00:00,817.44,818.41,817.86,816.79,817.16,817.64,817.94,817.20,458.82,38.53,39.99,78.52,3.49,16.28,1.07,2099.100,1079.458


# 欄位定義

In [170]:
action = df.columns[df.columns.str.contains('COT')].tolist()
value = ['乙烯產量','丙烯產量']
state = set(df.columns) - set(action_col) - set(value_col)
state = list(state)
action,value,state

(['H-110 COT',
  'H-120 COT',
  'H-130 COT',
  'H-140 COT',
  'H-150 COT',
  'H-160 COT',
  'H-170 COT',
  'H-180 COT',
  'H-190 COT'],
 ['乙烯產量', '丙烯產量'],
 ['O', 'N', 'i-P值', 'n+i  P值', 'n-P值', 'Aromatics'])

# 資料轉換

In [171]:
mm_state = MinMaxScaler()
mm_action = MinMaxScaler()
mm_value = MinMaxScaler()

df[state] = mm_state.fit_transform(df[state])
df[action] = mm_action.fit_transform(df[action])
df[value] = mm_value.fit_transform(df[value])

# 建立資料集 for 建模使用

In [172]:
def build_dataset(df,n=144): #一天24小時 操作為10分鐘一筆 一天1440分鐘 1440/10 =  144
    data = {
        'state':[],
        'action':[],
        'value':[],
        'idx':[],
        }
    
    for i in tqdm(range(0,len(df)-n,n)):
        curr = df.iloc[i:i+n] #一小段資料
        if np.isnan(curr.values).sum() == 0: #確保這段資料無缺失值
            data['state'].append(curr[state].values) # 這段時間所有狀態
            data['action'].append(curr[action].values) # 這段時間所有操作
            data['value'].append(curr[value].values[-1]) # 這段時間最後一個時間點的產量
            data['idx'].append(curr.index[-1]) #這段時間最後 出料對應的時間點
    
    # reshape
    data['state'] = np.array(data['state'])
    data['action'] = np.array(data['action'])
    data['value'] = np.array(data['value'])
    
    # store mm_scaler
    data['mm_state'] = mm_state
    data['mm_action'] = mm_action
    data['mm_value'] = mm_value
    
    # store col_name
    data['state_col'] = state
    data['action_col'] = action
    data['value_col'] = value
    
    return data

In [173]:
data = build_dataset(df.iloc[1:,:])

In [174]:
data['action'].shape,data['state'].shape,data['value'].shape

((365, 144, 9), (365, 144, 6), (365, 2))

In [175]:
np.isnan(data['action']).sum(),np.isnan(data['state']).sum(),np.isnan(data['value']).sum()

(0, 0, 0)

In [176]:
# 保存
import joblib
print(len(data['idx']))
joblib.dump(data,'../data/dataset.pkl')

365


['../data/dataset.pkl']